In [22]:
import requests
from lxml import etree
import time
import pymysql

loc="shanghai"
year = 2011
odd = [1,3,5,7,8,10,12]
even = [4,6,9,11]
num = 1
for year in range(2012,2016):
    for month in range(1,13):
        if month in odd:
            day = 31
        elif month in even:
            day = 30
        else:
            if (year%4 == 0) & (year%100 != 0):  
                day = 29 
            elif year%400 == 0:  
                day = 29  
            else:  
                day = 28
        if (month/10 < 1):
            newmonth = "0{}".format(month)
        else:
            newmonth = month            
        url = 'http://lishi.tianqi.com/shanghai/{}{}.html'.format(year,newmonth)
        data = requests.get(url).text
        for actday in range(1,day+1):  

            db = pymysql.connect("localhost","root","xxxx","forecast",use_unicode=True, charset="utf8")
            cursor = db.cursor()           
            s = etree.HTML(data)

            high_tem = s.xpath('//*[@id="tool_site"]/div[2]/ul[{}]/li[2]/text()'.format(actday+1))
            if high_tem == []:
                high = 0
            else:
                high = high_tem[0]
            low_tem = s.xpath('//*[@id="tool_site"]/div[2]/ul[{}]/li[3]/text()'.format(actday+1))
            if low_tem == []:
                low = 0
            else:
                low = low_tem[0]
            condition = s.xpath('//*[@id="tool_site"]/div[2]/ul[{}]/li[4]/text()'.format(actday+1))
            if condition == []:
                condition = ""
            else:
                con = condition[0]
            date = "{}年{}月{}日".format(year,month,actday)
            
            sql = """INSERT INTO shanghai values('{}','{}','{}','{}','{}')""".format(num,date,high,low,con)
            num = num+1
            try:
               # 执行sql语句
               cursor.execute(sql)
               # 提交到数据库执行
               db.commit()
            except:
                db.rollback()
            db.close()
